In [12]:
import os
import sys
import warnings
import logging
import torch
from omegaconf import OmegaConf

# Suppress warnings and logging
warnings.filterwarnings('ignore')
logging.disable(logging.WARNING)

# Core imports
from foundational_ssm.utils import get_default_config, device, load_yaml_config, get_train_val_loaders, get_dataset_config
from foundational_ssm.models import SSMNeuroModel
from foundational_ssm.loss import CombinedLoss
from foundational_ssm.trainer import train_mtm
from foundational_ssm.metrics import ValidationMetrics

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
config_dir = "/cs/student/projects1/ml/2024/mlaimon/foundational_ssm/configs"
full_config = OmegaConf.load(os.path.join(config_dir, "cmt.yaml"))
dataset_config = get_dataset_config(full_config.dataset.name, subjects=full_config.dataset.subjects)
train_dataset, train_loader, val_dataset, val_loader = get_train_val_loaders(dataset_config.)

KeyError: {'name': 'perich_miller_population_2018', 'subjects': ['c', 'm', 't'], 'batch_size': 128, 'root': '/cs/student/projects1/ml/2024/mlaimon/data/foundational_ssm/processed/'}

In [16]:
full_config.dataset.name

'perich_miller_population_2018'